# Practica 4: Regresión Parte 2
**Elaborado por: Luis Fernando Becerra,  BEDA Aprendizaje de Máquinas 2024-1S **

Vamos a probar dos modelos avanzados de regresion: Redes Neuronales y Bosques Aleatorios. Para esto usaremos los datos housing. Estos datos incluyen información sobre precio de la vivienda un determinado distrito de California y algunas estadísticas resumidas sobre ellas basadas en los datos del censo de 1990. Hay que tener en cuenta que los datos no están depurados, por lo que se requieren algunos pasos de preprocesamiento. Las caracteristicas incluidas son:

* longitude
* latitude
* housing_median_age
* total_rooms
* total_bedrooms
* population
* households
* median_income
* median_house_value
* ocean_proximity

## Lectura de datos y pre-procesamiento



In [3]:
#Importar librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
#Leer archivo housing.csv
dataset_dir = os.path.join(os.path.dirname(os.getcwd()),"datasets","housing.csv")
df = pd.read_csv(dataset_dir)

In [3]:
#Revisión de los datos
#Cabecera
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
#Información
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
#Eliminar muestras con valores perdidos
##
df.dropna(axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [6]:
#Conversión de variables categoricas
##

df = pd.get_dummies(df,columns=['ocean_proximity'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitude                   20433 non-null  float64
 1   latitude                    20433 non-null  float64
 2   housing_median_age          20433 non-null  float64
 3   total_rooms                 20433 non-null  float64
 4   total_bedrooms              20433 non-null  float64
 5   population                  20433 non-null  float64
 6   households                  20433 non-null  float64
 7   median_income               20433 non-null  float64
 8   median_house_value          20433 non-null  float64
 9   ocean_proximity_<1H OCEAN   20433 non-null  bool   
 10  ocean_proximity_INLAND      20433 non-null  bool   
 11  ocean_proximity_ISLAND      20433 non-null  bool   
 12  ocean_proximity_NEAR BAY    20433 non-null  bool   
 13  ocean_proximity_NEAR OCEAN  20433 no

##  División de los datos para entrenamiento

Nuestra variable objetivo es el valor de la vivienda ('median_house_value').

In [7]:
#Separamos entradas (X) y salidas (y)
##

y = df['median_house_value'].values
X = df.drop('median_house_value',axis=1).values
print(X.shape,y.shape)

(20433, 13) (20433,)


In [8]:
# Dividimos los datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(14303, 13) (6130, 13) (14303,) (6130,)


Vamos a normalizar los datos. Para ello utilizaremos el metodo min_max_scaler

In [9]:
from sklearn.preprocessing import MinMaxScaler
##

min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
x_test = min_max_scaler.transform(X_test)

## Perceptron Multicapa usando Keras

Keras es un API de alto nivel para Deep Learning (DL). Permite construir, entrenar, evaluar y ejecutar varias clases de redes neuronales.

Se recomienda consultar las especificaciones técnicas de la librería en el siguiente link.

Existen otros APIs para la implementación de técnicas de DL. Sin embargo, Keras ha ganado popularidad por su facilidad de uso, flexibilidad, y portabilidad.

TensorFlow incluye la librería Keras, facilitando la lectura y preprocesamiento de datos. En esta práctica usaremos esta librería tk.keras, aunque no usaremos características específicas de TensorFlow. Esta librería se encuentra preinstalada en colab.

Otra librería popular es PyTorch. Las versiones recientes de PyTorch y Keras son similares, en parte inspiradas por la facilidad de uso de Scikit-Learn. Por lo anterior, una vez se conoce Keras no debe ser difícil cambiar a PyTorch.

Paso 1
Vamos a construir una red neuronal multicapa

In [1]:
# Importacion de librerias
##
# Importacion de librerias
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout  
from keras.callbacks import EarlyStopping


Ahora, vamos a construir la red neuronal multicapa (MLP) empleando el modelo Sequential API de Keras. Este modelo permite crear una pila de capas simples.

Entre los tipos de capas que podemos crear se encuentra:


1.   Flatten: permite pasar datos matriciales a 1 dimensión. En este proceso se agrega una dimensión a cada muestra con un valor de 1s (para incluir el bias como un peso en el entrenamiento)
2.   Dense: capas de neuronas completamente conectada. Entre los parámetros se incluyen el número de neuronas y la función de activación. Los pesos se inicializan de forma aleatoria y el bias se inicializa en 0s. Las funciones de activacion soportadas las puede consultar en este [link](https://keras.io/api/layers/activations/).


In [10]:
#Construccion del MLP
##

model = Sequential()
model.add(Dense(1000,input_shape = (X_train.shape[1],),activation = 'relu'))
model.add(Dense(500,activation = 'relu'))
model.add(Dense(250,activation = "relu"))
model.add(Dense(1,activation='linear'))
model.summary()

c:\Users\jupja\OneDrive\Escritorio\IA\Regresion y clasificación\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1000)           │        14,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 250)            │       125,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           251 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 640,001 (2.44 MB)

 Trainable params: 640,001 (2.44 MB)

 Non-trainable params: 0 (0.00 B)

Nota: cada capa incluye un gran número de parámetros. Por ejemplo, la primera capa oculta tiene 13x1000 pesos que estimar, más 1000 pesos relacionados con el bias. Esto implica que en total debemos entrenar para esta capa 14,000 parámetros. Esto permite una alta flexibilidad para ajustarse a los datos de entrenamiento, pero debido al tamaño del modelo puede tener un sobreajuste.

Exploremos un poco las capas del modelo:

In [ ]:
#Podemos acceder a la lista de capas:
##
model.layers

In [ ]:
#Revisar los pesos y bias de cada capa
hidden1 = model.layers[1]
print(hidden1.name)
weights, biases = hidden1.get_weights()
#Mostrar los pesos
weights

In [ ]:
#Mostrar biases
biases

Note que la capa se inicializa con pesos aleatorios y bias en 0s. Keras incluye otros métodos de inicialización de
los pesos. El tamaño de la primera capa oculta depende del número de entradas, por eso se recomienda incluir este número en la capa inicial.

Una vez creado el modelo, se invoca el método compile() que permite especificar:

*   La función de perdida: Keras incluye una gran variedad de funciones de perdida. Por ejemplo, diferentes tipos de entropía cruzada para clasificación, tipos de error como MSE y MAE para regresión, entre otros. Consulte las posibles funciones de perdida en este [link](https://keras.io/api/losses/).
*   El método de optimización: los optimizadores disponibles se pueden consultar en este [link](https://keras.io/api/optimizers/). El optimizar es el método empleado durante el algoritmo de backpropagation. En esta práctica usaremos SGD (Gradiente Estocástico Descendiente), inicialmente con sus parámetros por defecto (paso de aprendizaje = 0.01).
*   La métrica de evaluación: en este caso usaremos  el MSE. Otras métricas disponibles pueden consultarse en el siguiente [link](https://keras.io/api/metrics/).


In [ ]:
#Vamos a compilar el modelo
model.compile(optimizer='Adagrad', loss='mse', metrics=['mae'])

In [ ]:
#Una vez compilado, esta listo para ser entrenado
#a traves del metodo fit
#Le enviamos los datos de entrenamiento, el numero
#de epocas, y el set de validacion (opcional)
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    epochs=30,
                    batch_size=50,
                    verbose=1)

Notas:

*   En vez de enviar los datos de validación, también puede especificar el porcentaje de datos a usar de validación. Por ejemplo, validation_split=0.1 indicara a Keras que use el 10% de los datos antes de aleatorizarlos para validación
*   El método fit() retorna un objeto con la historia del entrenamiento, incluyendo los parámetros entrenados, la lista de épocas con las métricas de desempeño seleccionada. Podemos usar estos datos para graficar la curva de aprendizaje del proceso.

In [ ]:
#Curva de aprendizaje
history_dict = history.history
loss_values = history_dict['loss'] # puedes modificar esto
val_loss_values = history_dict['val_loss'] # puedes modificar esto
epochs = range(1, len(loss_values) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Vamos a incluir EarlyStopping para evitar el sobreajuste y además a aumentar el número de epocas: permite detener el proceso cuando la función de perdida ya no esta decreciendo. El parametro patience indica el número de epocas a considerar cuando ya no hay una mejora del entrenamiento

In [ ]:
#Configurar EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=10,
                   restore_best_weights = True)

In [ ]:
#Entrenamos el modelo
#Tomara unos minutos
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    callbacks=[es],
                    epochs=100,
                    batch_size=50,
                    verbose=1)

In [ ]:
# specificar las dimensiones
fig, axes = plt.subplots(1,2) # 1 row, 2 columns

# resultados de entrenamiento
axes[0].scatter(x=y_train, y=model.predict(X_train))
axes[0].set_xlabel("Actual", fontsize=10)
axes[0].set_ylabel("Predicted",  fontsize=10)
axes[0].set_title("Training")

x = np.linspace(*axes[0].get_xlim())
axes[0].plot(x, x, color='red')
# resultados de validación
axes[1].scatter(x=y_test, y=model.predict(X_test))
axes[1].set_xlabel("Actual", fontsize=10)
axes[1].set_ylabel("Predicted",  fontsize=10)
axes[1].set_title("Validation")

x = np.linspace(*axes[1].get_xlim())
axes[1].plot(x, x, color='red')


fig.tight_layout()
plt.show()

** Realiza las Métricas de error de regresión. **

In [ ]:
#Metricas de desempeño
from sklearn import metrics

pred = model.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('R2_score:', metrics.r2_score(y_test, pred))


192/192 [==============================] - 1s 4ms/step
MAE: 63379.46696892843
MSE: 6963769341.709494
RMSE: 83449.20216340893
R2_score: 0.48773252202176165


# ¡Gran trabajo!